In [1]:
import pandas as pd

In [109]:
    def get_train_test_data(year = 2018):
    train_data = pd.read_csv('../ouput/train_2018.csv')

    train_data['WTseed']= train_data['WTseed'].fillna(18)####
    train_data['LTseed']= train_data['LTseed'].fillna(18)####

    train_data['ID'] = train_data.apply(lambda r: '_'.join(map(str, [r['Season']]+sorted([r['WTeamID'],r['LTeamID']]))), axis=1)
    train_data['IDTeams'] = train_data.apply(lambda r: '_'.join(map(str, sorted([r['WTeamID'],r['LTeamID']]))), axis=1)
    train_data['Team1'] = train_data.apply(lambda r: sorted([r['WTeamID'],r['LTeamID']])[0], axis=1)
    train_data['Team2'] = train_data.apply(lambda r: sorted([r['WTeamID'],r['LTeamID']])[1], axis=1)
    train_data['IDTeam1'] = train_data.apply(lambda r: '_'.join(map(str, [r['Season'], r['Team1']])), axis=1)
    train_data['IDTeam2'] = train_data.apply(lambda r: '_'.join(map(str, [r['Season'], r['Team2']])), axis=1)
    train_data['Pred'] = train_data.apply(lambda r: 1. if sorted([r['WTeamID'],r['LTeamID']])[0]==r['WTeamID'] else 0., axis=1)
    train_data['ScoreDiff'] = train_data['WScore']-train_data['LScore']
    train_data['ScoreDiff'] = train_data.apply(lambda r: r['ScoreDiff'] * -1 if r['Pred'] == 0. else r['ScoreDiff'], axis=1)
    train_data['Team1Score'] = train_data.apply(lambda r: r['WScore'] if r['Pred'] == 1. else r['LScore'],axis=1)
    train_data['Team2Score'] = train_data.apply(lambda r: r['WScore'] if r['Pred'] == 0. else r['LScore'],axis=1)
    train_data['Team1Seed'] = train_data.apply(lambda r: r['WTseed']if r['Pred'] == 1. else r['LTseed'],axis=1)
    train_data['Team2Seed'] = train_data.apply(lambda r: r['WTseed']if r['Pred'] == 0. else r['LTseed'],axis=1)
    train_data['SeedDiff'] = train_data.apply(lambda r: r['Team1Seed']-r['Team2Seed'],axis = 1)
    train_data['Team1ScoreRatio'] = train_data['Team1Score']/(train_data['Team1Score']+train_data['Team2Score'])
    train_data['Team2ScoreRatio'] = train_data['Team2Score']/(train_data['Team1Score']+train_data['Team2Score'])

    train_data = train_data.drop(['index','DayNum','WScore','LScore','WTseed','LTseed','WTeamID','LTeamID'],axis=1)

    train_data.head()

    team1d_score_spread = train_data.groupby(['Season', 'Team1'])[['ScoreDiff', 'Team1Score','Team1ScoreRatio']].mean().reset_index()\
    .set_index('Season').rename(columns = {'ScoreDiff' : 'ScoreDiff1','Team1ScoreRatio':'Team1ScoreRatio_avg','Team1Score':'Team1Score_avg'})
    team2d_score_spread = train_data.groupby(['Season', 'Team2'])[['ScoreDiff', 'Team2Score','Team2ScoreRatio']].mean().reset_index()\
    .set_index('Season').rename(columns = {'ScoreDiff' : 'ScoreDiff2','Team2ScoreRatio':'Team2ScoreRatio_avg','Team2Score':'Team2Score_avg'})
    score_spread = team1d_score_spread.join(team2d_score_spread).reset_index()

    X_train = pd.merge(train_data,score_spread,on=['Team1','Team2','Season'])

    y_train = X_train['Pred']
    X_train = X_train.drop(['Pred','Team1Score','Team2Score','Team1ScoreRatio','Team2ScoreRatio','ScoreDiff'],axis=1)
return X_train,y_train

0        1.0
1        1.0
2        1.0
3        1.0
4        1.0
5        1.0
6        1.0
7        1.0
8        1.0
9        1.0
10       1.0
11       1.0
12       1.0
13       1.0
14       1.0
15       1.0
16       0.0
17       1.0
18       0.0
19       0.0
20       1.0
21       1.0
22       1.0
23       0.0
24       1.0
25       1.0
26       1.0
27       0.0
28       1.0
29       0.0
        ... 
15591    0.0
15592    0.0
15593    1.0
15594    1.0
15595    1.0
15596    1.0
15597    0.0
15598    0.0
15599    0.0
15600    0.0
15601    1.0
15602    1.0
15603    1.0
15604    0.0
15605    0.0
15606    1.0
15607    0.0
15608    0.0
15609    1.0
15610    1.0
15611    0.0
15612    0.0
15613    1.0
15614    0.0
15615    0.0
15616    1.0
15617    0.0
15618    1.0
15619    1.0
15620    0.0
Name: Pred, Length: 15621, dtype: float64